In [ ]:
# You can add more model configs like below.
MODELS_CONFIG = {
    'faster_rcnn_r50_fpn_1x': {
        'config_file': 'configs/faster_rcnn_r50_fpn_1x.py'
    },
    'cascade_rcnn_r50_fpn_1x': {
        'config_file': 'configs/cascade_rcnn_r50_fpn_1x.py',
    },
    'cascade_rcnn_r101_fpn_1x': {
        'config_file': 'configs/cascade_rcnn_r101_fpn_1x.py',
    },
    'libra_faster_rcnn_r101_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_faster_rcnn_r101_fpn_1x.py',
    },
    'libra_faster_rcnn_r50_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_faster_rcnn_r50_fpn_1x.py',
    },
    'libra_faster_rcnn_x101_64x4d_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_faster_rcnn_x101_64x4d_fpn_1x.py',
    },
    'libra_fast_rcnn_r50_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_fast_rcnn_r50_fpn_1x.py',
    },
    'libra_retinanet_r50_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_retinanet_r50_fpn_1x.py',
    },
    'retinanet_r50_fpn_1x': {
        'config_file': 'configs/retinanet_r50_fpn_1x.py',
    }
}

In [ ]:
!unzip /content/mmdetection_object_detection_demo.zip

In [ ]:
# TODO: change URL to your fork of my repository if necessary.
git_repo_url = 'https://github.com/Tony607/mmdetection_object_detection_demo'

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
# selected_model = 'libra_faster_rcnn_r101_fpn_1x'  # 'cascade_rcnn_r50_fpn_1x'

selected_model = 'cascade_rcnn_r101_fpn_1x' 

# Total training epochs.
total_epochs = 1

# Name of the config file.
config_file = MODELS_CONFIG[selected_model]['config_file']

In [ ]:
pip install torch==1.1.0 torchvision==0.3.0

In [ ]:
import os
from os.path import exists, join, basename, splitext

%cd /content
project_name = os.path.abspath(splitext(basename(git_repo_url))[0])
# print(project_name)
mmdetection_dir = os.path.join(project_name, "mmdetection")
if not exists(project_name):
    # clone "depth 1" will only get the latest copy of the relevant files.
    # !git clone -q --recurse-submodules --depth 1 $git_repo_url
    print("Update mmdetection repo")
    !cd {mmdetection_dir} 
        # dependencies
    !pip install -q mmcv terminaltables
        # build
    !cd {mmdetection_dir} && python setup.py install
    !pip install -r {os.path.join(mmdetection_dir, "requirements.txt")}


import sys
sys.path.append(mmdetection_dir)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

/content


In [ ]:
# !rm -rf /content/mmdetection_object_detection_demo/data/VOC2007
# !rm -rf /content/mmdetection_object_detection_demo/mmdetection/data/VOCdevkit/VOC2007
# !rm -rf /content/mmdetection_object_detection_demo/mmdetection/work_dirs/libra_faster_rcnn_x101_64x4d_fpn_1x
# !rm -rf /content/mmdetection_object_detection_demo
# !rm -rf /content/mmdetection_object_detection_demo/mmdetection/Annotations/JPEGImages
# !rm -rf /content/mmdetection_object_detection_demo/mmdetection/Annotations/ImageSets
# !rm -rf /content/mmdetection_object_detection_demo/mmdetection/Annotations/Annotations

# !cp '/content/drive/My Drive/tumor/ImageSets.zip' '/content'
# !unzip '/content/drive/My Drive/mmdetect/Annotations.zip'
!unzip '/content/drive/My Drive/tumor/ImageSets.zip'
# !unzip '/content/drive/My Drive/mmdetect/mmdetection_object_detection_demo.zip'
# !zip -r '/content/folder.zip' '/content/mmdetection_object_detection_demo'

# !mv '/content/folder.zip' '/content/drive/My Drive/mmdetect'

!mv '/content/ImageSets' '/content/mmdetection_object_detection_demo/data/VOC2007'
!mv '/content/JPEGImages' '/content/mmdetection_object_detection_demo/data/VOC2007'
!mv '/content/Annotations' '/content/mmdetection_object_detection_demo/data/VOC2007'

In [ ]:
# !mv '/content/mmdetection_object_detection_demo/data/VOC2007/Main' '/content/mmdetection_object_detection_demo/data/VOC2007/ImageSets'
!mv '/content/mmdetection_object_detection_demo/data/VOC2007/Main/trainval.txt' '/content/mmdetection_object_detection_demo/data/VOC2007/ImageSets/Main'

In [ ]:
%cd {project_name}

/content/mmdetection_object_detection_demo


In [ ]:
import os
import glob

import pandas as pd8
import xml.etree.ElementTree as ET

In [ ]:
anno_path = os.path.join(project_name, "data/VOC2007/Annotations")
voc_file = os.path.join(mmdetection_dir, "mmdet/datasets/voc.py")

In [ ]:
classes_names = []
xml_list = []
for xml_file in glob.glob(anno_path + "/*.xml"):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall("object"):
        classes_names.append(member[0].text)
classes_names = list(set(classes_names))
classes_names.sort()
classes_names

['tumor']

In [ ]:
import re

fname = voc_file
with open(fname) as f:
  
    s = f.read()
    s = re.sub('CLASSES = \(.*?\)',
               'CLASSES = ({})'.format(", ".join(["\'{}\'".format(name) for name in classes_names])), s, flags=re.S)
with open(fname, 'w') as f:
    f.write(s)
!cat {voc_file}

from mmdet.core import eval_map, eval_recalls
from .registry import DATASETS
from .xml_style import XMLDataset


@DATASETS.register_module
class VOCDataset(XMLDataset):

    CLASSES = ('tumor')

    def __init__(self, **kwargs):
        super(VOCDataset, self).__init__(**kwargs)
        if 'VOC2007' in self.img_prefix:
            self.year = 2007
        elif 'VOC2012' in self.img_prefix:
            self.year = 2012
        else:
            raise ValueError('Cannot infer dataset year from img_prefix')

    def evaluate(self,
                 results,
                 metric='mAP',
                 logger=None,
                 proposal_nums=(100, 300, 1000),
                 iou_thr=0.5,
                 scale_ranges=None):
        if not isinstance(metric, str):
            assert len(metric) == 1
            metric = metric[0]
        allowed_metrics = ['mAP', 'recall']
        if metric not in allowed_metrics:
            raise KeyError('metric {} is not supported'.format(metric)

In [ ]:
import os
config_fname = os.path.join(project_name, 'mmdetection', config_file)

assert os.path.isfile(config_fname), '`{}` not exist'.format(config_fname)
config_fname

'/content/mmdetection_object_detection_demo/mmdetection/configs/cascade_rcnn_r101_fpn_1x.py'

In [ ]:
import re
fname = config_fname
classes_names = ['tumor']
with open(fname) as f:
    s = f.read()
    work_dir = re.findall(r"work_dir = \'(.*?)\'", s)[0]
    
    # Update `num_classes` including `background` class.
    s = re.sub('num_classes=.*?,',
               'num_classes={},'.format(len(classes_names) + 1), s)
    s = re.sub('ann_file=.*?\],',
               "ann_file=data_root + 'VOC2007/ImageSets/Main/trainval.txt',", s, flags=re.S)
    s = re.sub('total_epochs = \d+',
               'total_epochs = {} #'.format(total_epochs), s)
    if "CocoDataset" in s:
        s = re.sub("dataset_type = 'CocoDataset'",
                   "dataset_type = 'VOCDataset'", s)
        s = re.sub("data_root = 'data/coco/'",
                   "data_root = 'data/VOCdevkit/'", s)
        s = re.sub("annotations/instances_train2017.json",
                   "VOC2007/ImageSets/Main/trainval.txt", s)
        s = re.sub("annotations/instances_val2017.json",
                   "VOC2007/ImageSets/Main/test.txt", s)
        s = re.sub("annotations/instances_val2017.json",
                   "VOC2007/ImageSets/Main/test.txt", s)
        s = re.sub("train2017", "VOC2007", s)
        s = re.sub("val2017", "VOC2007", s)
    else:
        s = re.sub('img_prefix=.*?\],',
                   "img_prefix=data_root + 'VOC2007/',".format(total_epochs), s)
with open(fname, 'w') as f:
    f.write(s)

!cat {config_fname}

# model settings
model = dict(
    type='CascadeRCNN',
    num_stages=3,
    pretrained='torchvision://resnet101',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_scales=[8],
        anchor_ratios=[0.5, 1.0, 2.0],
        anchor_strides=[4, 8, 16, 32, 64],
        target_means=[.0, .0, .0, .0],
        target_stds=[1.0, 1.0, 1.0, 1.0],
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='SmoothL1Loss', beta=1.0 / 9.0, loss_weight=1.0)),
    bbox_roi_extractor=dict(
        type='SingleRoIExtractor',
        roi_l

In [ ]:
%cd {mmdetection_dir}
!python setup.py install

/content/mmdetection_object_detection_demo/mmdetection
running install
running bdist_egg
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying mmdet/version.py -> build/lib.linux-x86_64-3.6/mmdet
copying mmdet/datasets/voc.py -> build/lib.linux-x86_64-3.6/mmdet/datasets
running build_ext
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mmdet
creating build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib.linux-x86_64-3.6/mmdet/utils/collect_env.py -> build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib.linux-x86_64-3.6/mmdet/utils/util_mixins.py -> build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib.linux-x86_64-3.6/mmdet/utils/_

In [ ]:
os.makedirs("data/VOCdevkit", exist_ok=True)
voc2007_dir = os.path.join(project_name, "data/VOC2007")
os.system("ln -s {} data/VOCdevkit".format(voc2007_dir))

0

In [ ]:
# /content/mmdetection_object_detection_demo/mmdetection/mmdet/datasets/pipelines/loading.py
        if 'mitosis' in filename:
            pass
        elif 'H' in filename or 'A' in filename:
            filename = filename.replace('.jpg','.bmp')
        else:
            filename = filename.replace('.jpg','.tif') 

In [ ]:
!mv '/content/mmdetection_object_detection_demo/mmdetection/data/VOCdevkit/VOC2007/ImageSets/trainval.txt' '/content/mmdetection_object_detection_demo/mmdetection/data/VOCdevkit/VOC2007/ImageSets/Main'

In [ ]:
!python tools/train.py {config_fname}

2020-05-29 15:53:56,157 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.6.9 (default, Apr 18 2020, 01:56:04) [GCC 8.4.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.1, V10.1.243
GPU 0: Tesla P4
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.1.0
PyTorch compiling details: PyTorch built with:
  - GCC 4.9
  - Intel(R) Math Kernel Library Version 2018.0.1 Product Build 20171007 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v0.18.1 (Git Hash 7de7e5d02bf687f971e7668963649728356e0c20)
  - OpenMP 201307 (a.k.a. OpenMP 4.0)
  - NNPACK is enabled
  - CUDA Runtime 9.0
  - NVCC architecture flags: -gencode;arch=compute_35,code=sm_35;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_50,code=compute_50
  - CuDNN 7.5.1
  - Magma 2.5.0
  - Build settings: BLAS=MKL, BUILD_TYPE=

In [ ]:
checkpoint_file = os.path.join(mmdetection_dir, work_dir, "latest.pth")
assert os.path.isfile(
    checkpoint_file), '`{}` not exist'.format(checkpoint_file)
checkpoint_file

'/content/mmdetection_object_detection_demo/mmdetection/./work_dirs/cascade_rcnn_r101_fpn_1x/latest.pth'

In [ ]:
# !rm -rf '/content/mmdetection_object_detection_demo/mmdetection/work_dirs/cascade_rcnn_r101_fpn_1x'

In [ ]:
%cd {mmdetection_dir}
!python setup.py develop

/content/mmdetection_object_detection_demo/mmdetection
running develop
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.6/mmdet/ops/utils/compiling_info.cpython-36m-x86_64-linux-gnu.so -> mmdet/ops/utils
copying build/lib.linux-x86_64-3.6/mmdet/ops/nms/nms_cpu.cpython-36m-x86_64-linux-gnu.so -> mmdet/ops/nms
copying build/lib.linux-x86_64-3.6/mmdet/ops/nms/nms_cuda.cpython-36m-x86_64-linux-gnu.so -> mmdet/ops/nms
copying build/lib.linux-x86_64-3.6/mmdet/ops/roi_align/roi_align_cuda.cpython-36m-x86_64-linux-gnu.so -> mmdet/ops/roi_align
copying build/lib.linux-x86_64-3.6/mmdet/ops/roi_pool/roi_pool_cuda.cpython-36m-x86_64-linux-gnu.so -> mmdet/ops/roi_pool
copying build/lib.linux-x86_64-3.6/mmdet/ops/dcn/deform_conv_c

In [ ]:
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

import mmcv
from mmcv.runner import load_checkpoint
import mmcv.visualization.image as mmcv_image
# fix for colab


def imshow(img, win_name='', wait_time=0): plt.figure(
    figsize=(10, 10)); plt.imshow(img)


mmcv_image.imshow = imshow
from mmdet.models import build_detector
from mmdet.apis import inference_detector, show_result, init_detector

In [ ]:
# !unzip '/content/drive/My Drive/tumor/val.zip' 
# !unzip '/content/drive/My Drive/mmdetect/Annotations1.zip' 
# !unzip '/content/drive/My Drive/tumor/test.zip'

Archive:  /content/drive/My Drive/mmdetect/Annotations1.zip
  inflating: Annotations1/27_01_126.xml  
  inflating: Annotations1/27_01_86.xml  
  inflating: Annotations1/30_01_4.xml  
  inflating: Annotations1/32_01_74.xml  
  inflating: Annotations1/32_01_94.xml  
  inflating: Annotations1/36_01_114.xml  
  inflating: Annotations1/36_01_32.xml  
  inflating: Annotations1/36_01_5.xml  
  inflating: Annotations1/36_01_76.xml  
  inflating: Annotations1/41_01_103.xml  
  inflating: Annotations1/41_01_55.xml  
  inflating: Annotations1/41_01_58.xml  
  inflating: Annotations1/41_01_79.xml  
  inflating: Annotations1/45_01_36.xml  
  inflating: Annotations1/45_01_50.xml  
  inflating: Annotations1/45_01_7.xml  
  inflating: Annotations1/45_01_9.xml  
  inflating: Annotations1/50_01_108.xml  
  inflating: Annotations1/50_01_109.xml  
  inflating: Annotations1/50_01_123.xml  
  inflating: Annotations1/50_01_130.xml  
  inflating: Annotations1/50_01_37.xml  
  inflating: Annotations1/50_01_38.

In [ ]:
import csv
import numpy as np
import xml.etree.ElementTree as ET
import math
from tqdm import tqdm
import cv2
%cd {mmdetection_dir}

score_thr = 0.7

# build the model from a config file and a checkpoint file
model = init_detector(config_fname, '/content/drive/My Drive/tumor/latest1.pth')

direc = '/content/mmdetection_object_detection_demo/mmdetection/data/VOCdevkit/VOC2007/'
# for d in tqdm(dataset_dicts):

TP=0
FN=0
FP=0

true_count = 0
false_count = 0

for image in tqdm(os.listdir('/content/mmdetection_object_detection_demo/mmdetection/data/VOCdevkit/VOC2007/JPEGImages')):
    try:
        
        # image = 'A03_04Ac_mitosis_1.jpg'
        # print(image)
        name = image.split('.')
        
        image = direc + 'JPEGImages/' + image  
        anno = direc + 'Annotations/' + name[0] + '.xml'

        results = inference_detector(model, image)
        
        im = cv2.imread(image)

        tree = ET.parse(anno)
        root = tree.getroot()


        bbox_center_x = []
        bbox_center_y = []


        predicted_centers_x = []
        predicted_centers_y = []
        
        bboxs = []
        for elem in root:
            for subelem in elem:
                bbox = []
                for subsubelem in subelem:
                  bbox.append(float(subsubelem.text))
                if bbox != [] :
                    bboxs.append(bbox)


        predicted_bboxs = results[0]
        predicted_bboxs = np.array(predicted_bboxs)
        

        for bbox in bboxs:
            bbox_center_x.append((bbox[0]+bbox[2])/2)
            bbox_center_y.append((bbox[1]+bbox[3])/2)
                
        for predicted_bbox in predicted_bboxs:
            predicted_centers_x.append((predicted_bbox[0]+predicted_bbox[2])/2)
            predicted_centers_y.append((predicted_bbox[1]+predicted_bbox[3])/2)


        temp_bbox_x = bbox_center_x
        temp_bbox_y = bbox_center_y


        i = 0
        while i < len(predicted_centers_x):
            j = 0
            while j < len(predicted_centers_x):
                if i == j:
                    j += 1
                    pass
                else:
                    
                    if (math.sqrt( (predicted_centers_y[j] - predicted_centers_y[i])**2 + 
                            (predicted_centers_x[j] - predicted_centers_x[i])**2 ) ) <= 30:

                        predicted_centers_x = np.delete(predicted_centers_x,j)
                        predicted_centers_y = np.delete(predicted_centers_y,j)

                    else:
                        j += 1

            print('\n')
            i += 1
        

        for i in range(len(predicted_centers_x)):
            for j in range(len(temp_bbox_x)):
                if (math.sqrt( (temp_bbox_x[j] - predicted_centers_x[i])**2 + 
                    (temp_bbox_y[j] - predicted_centers_y[i])**2 ) ) <= 30:
                    TP += 1
                    
                    try:
                        cv2.imwrite('/content/cascade_tp/tp_'+str(true_count)+'.jpg', im[int(predicted_centers_y[i]-32):int(predicted_centers_y[i]+32), int(predicted_centers_x[i]-32):int(predicted_centers_x[i]+32)])
                    except Exception as e:
                        print(e)
                    true_count = true_count+1
                    break
                if j == ( len(temp_bbox_x)-1 ):
                    FP += 1
                    try:
                        cv2.imwrite('/content/cascade_fp/fp_'+str(false_count)+'.jpg', im[int(predicted_centers_y[i]-32):int(predicted_centers_y[i]+32), int(predicted_centers_x[i]-32):int(predicted_centers_x[i]+32)])
                    except Exception as e:
                        print(e)                    
                    false_count = false_count+1
        # break
    except Exception as e:
        print(e)

print('TP : ',TP)
print('FP : ',FP)

# Precision = (TP/(TP+FP))
# Recall = (TP/(len(d['annotations'])))

# f_score = 2*(Recall * Precision) / (Recall + Precision)
# print(f_score)




/content/mmdetection_object_detection_demo/mmdetection


  0%|          | 1/2231 [00:00<08:39,  4.29it/s]

  0%|          | 2/2231 [00:00<08:21,  4.45it/s]

  0%|          | 3/2231 [00:00<08:05,  4.59it/s]

  0%|          | 5/2231 [00:01<07:52,  4.71it/s]

...

100%|█████████▉| 2228/2231 [07:42<00:00,  4.68it/s]

100%|█████████▉| 2229/2231 [07:42<00:00,  4.71it/s]

100%|█████████▉| 2230/2231 [07:42<00:00,  4.71it/s]

100%|██████████| 2231/2231 [07:43<00:00,  4.82it/s]



TP :  2536
FP :  810


In [ ]:
# !cp -r '/content/cascade_tp' '/content/drive/My Drive/tumor/train_tp'
!cp -r '/content/cascade_fp' '/content/drive/My Drive/tumor/train_fp'

In [ ]:
Precision = (TP/(TP+FP))
Recall = (TP/(TP+FN))

f_score = 2*(Recall * Precision) / (Recall + Precision)
print(f_score)

In [ ]:
0.8735988963614416
0.7170431211498973 test
0.8416666666666668 val



In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('/content/mmdetection_object_detection_demo/data/VOC2007/Annotations/A03_04Ac_mitosis_1.xml')
root = tree.getroot()

bboxs = []
for elem in root:
    for subelem in elem:
        bbox = []
        for subsubelem in subelem:
          bbox.append(float(subsubelem.text))
        if bbox != [] :
            bboxs.append(bbox)

print(bboxs)

In [ ]:
import numpy as np
%cd {mmdetection_dir}

score_thr = 0.7

# build the model from a config file and a checkpoint file
model = init_detector(config_fname, '/content/mmdetection_object_detection_demo/mmdetection/work_dirs/cascade_rcnn_r101_fpn_1x/latest5.pth')

img = '/content/mmdetection_object_detection_demo/data/VOC2007/JPEGImages/A03_04Ac_mitosis_1.jpg'
results = inference_detector(model, img)
print(results)
# predicted_bboxs = results[0]
# predicted_bboxs = np.array(predicted_bboxs)
# # predicted_bboxs = np.delete(predicted_bboxs, (4), axis=0)
# print(predicted_bboxs)

show_result(img, results, model.CLASSES,
            score_thr=0.5, out_file="result.jpg")
from IPython.display import Image
Image(filename='result.jpg')